# ✈️ SwiftPlan: AI Travel Concierge

## Google x Kaggle AI Agents Intensive Capstone Project

**Track:** Concierge Agents

**Author:** Ratheshan Sathiyamoorthy

### Project Overview
SwiftPlan is a Multi-Agent System designed to eliminate the stress of travel planning. It automates the research and scheduling process, creating personalized, budget-conscious itineraries in seconds.

### Architecture & Key Concepts
This project demonstrates three key concepts from the course:

1.  **Multi-Agent System (Sequential):** * **Research Agent:** Uses Google Search (Grounding) to find real-time data on weather, attractions, and local culture.
    * **Planner Agent:** A reasoning engine that structures the raw data into a logical day-by-day plan.
    
2.  **Tools (Hybrid):** * **Google Search:** For external knowledge.
    * **Custom Python Tool (`currency_converter`):** A defined function that the agent actively calls to calculate budgets in local currency.

3.  **Observability & Memory:** * **Session Memory:** Maintains context across the research and planning phases.
    * **Logging:** Detailed console logs track the agent's thought process, tool usage, and execution status.

### How to Run
1.  Add your `GOOGLE_API_KEY` in the **Add-ons -> Secrets** menu.
2.  Run all cells.
3.  Use the `run_swiftplan()` function at the bottom to test different destinations!

### Setup & Configuration

In [11]:
# Installing necessary libraries
!pip install -q -U google-genai
!pip install -q rich

In [24]:
# Imports & Configuration
import os
import time
import logging
from google import genai
from google.genai import types
from rich.console import Console
from rich.markdown import Markdown

# Setup Observability (Logging) - Requirements Checklist [x]
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(message)s')
logger = logging.getLogger("SwiftPlan")

console = Console()

# Setup Client
try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    GEMINI_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")
    client = genai.Client(api_key=GEMINI_API_KEY)
    console.print("[green]SUCCESS: API Key loaded and Client initialized![/green]")
except Exception as e:
    console.print(f"[red]ERROR: Could not load API Key. Check Add-ons -> Secrets.[/red]\n{e}")

SUCCESS: API Key loaded and Client initialized!

### Define Tools
1. Google Search: For real-time travel info.
2. Currency Converter: A custom Python function that the Planner Agent can use to estimate trip costs.

In [25]:
# Tool 1: Google Search (Built-in)
google_search_tool = types.Tool(
    google_search=types.GoogleSearch()
)

# Tool 2: Custom Currency Converter
# First, define the actual Python function
def currency_converter(amount: float, from_currency: str, to_currency: str) -> str:
    """
    Converts a monetary amount from one currency to another using fixed estimated rates.
    """
    rates = {
        "USD": 1.0, "EUR": 0.95, "JPY": 150.0, "GBP": 0.79, "LKR": 300.0
    }
    
    from_curr = from_currency.upper()
    to_curr = to_currency.upper()
    
    if from_curr not in rates or to_curr not in rates:
        return f"Error: Exchange rate not available for {from_curr} to {to_curr}."
    
    # Calculate
    amount_in_usd = amount / rates[from_curr]
    final_amount = amount_in_usd * rates[to_curr]
    
    return f"{amount} {from_curr} is approximately {final_amount:.2f} {to_curr}"

# Next, create the Tool object with a manual FunctionDeclaration
# We explicitly define the parameters so the model knows how to use it.
currency_tool = types.Tool(
    function_declarations=[
        types.FunctionDeclaration(
            name="currency_converter",
            description="Converts a monetary amount from one currency to another.",
            parameters=types.Schema(
                type="OBJECT",
                properties={
                    "amount": types.Schema(
                        type="NUMBER", 
                        description="The amount of money to convert"
                    ),
                    "from_currency": types.Schema(
                        type="STRING", 
                        description="The currency code to convert from (e.g. USD)"
                    ),
                    "to_currency": types.Schema(
                        type="STRING", 
                        description="The currency code to convert to (e.g. JPY)"
                    ),
                },
                required=["amount", "from_currency", "to_currency"]
            )
        )
    ]
)

console.print("[blue]Tools Configured: Google Search & Currency Converter (Schema Fixed)[/blue]")

Tools Configured: Google Search & Currency Converter (Schema Fixed)

### The Agent Class
Memory: It has self.history to remember the conversation.

Retry Logic: It automatically retries if it hits a rate limit.

Observability: It logs every step to the console.

In [26]:

class Agent:
    def __init__(self, name, model, system_instruction, tools=None, tool_map=None):
        self.name = name
        self.model = model
        self.system_instruction = system_instruction
        self.tools = tools
        self.tool_map = tool_map or {} # Dictionary mapping tool names to actual Python functions
        self.history = [] 

    def generate_response(self, prompt, context=None):
        # 1. Prepare the Initial Prompt
        full_prompt = prompt
        if context:
            full_prompt = f"PREVIOUS CONTEXT:\n{context}\n\nCURRENT TASK:\n{prompt}"

        # Add the user's prompt to history
        self.history.append(types.Content(role="user", parts=[types.Part(text=full_prompt)]))

        console.print(f"\n[bold yellow]🤖 {self.name} is working...[/bold yellow]")

        # 2. The Execution Loop (Handles Tool Calls)
        # We loop to allow multiple tool calls if needed
        max_turns = 5
        current_turn = 0
        
        while current_turn < max_turns:
            current_turn += 1
            
            try:
                # Call the Model
                config = types.GenerateContentConfig(
                    system_instruction=self.system_instruction,
                    tools=self.tools,
                    temperature=0.7
                )

                response = client.models.generate_content(
                    model=self.model,
                    contents=self.history, # Send full conversation history
                    config=config
                )

                # Check if the model returned any content
                if not response.candidates:
                    return "Error: No response from model."
                
                # Get the content part
                part = response.candidates[0].content.parts[0]
                
                # CASE A: The Model wants to call a Function (Tool Use)
                if part.function_call:
                    fc = part.function_call
                    fn_name = fc.name
                    fn_args = fc.args
                    
                    console.print(f"[dim]   ⚙️ calling tool: {fn_name}({fn_args})...[/dim]")
                    
                    # 1. Execute the Python Function
                    if fn_name in self.tool_map:
                        tool_result = self.tool_map[fn_name](**fn_args)
                    else:
                        tool_result = f"Error: Tool '{fn_name}' not found."

                    # 2. Add the Model's "Call" to history
                    self.history.append(response.candidates[0].content)

                    # 3. Add the "Result" to history (Send it back to the model)
                    # We create a FunctionResponse part
                    fn_response_part = types.Part.from_function_response(
                        name=fn_name,
                        response={"result": tool_result}
                    )
                    self.history.append(types.Content(role="user", parts=[fn_response_part]))
                    
                    # Loop continues... model will now read the result and generate text.
                    continue

                # CASE B: The Model returned Text (Final Answer)
                else:
                    response_text = part.text
                    # Add final answer to history
                    self.history.append(response.candidates[0].content)
                    
                    logger.info(f"Agent {self.name} finished successfully.")
                    return response_text

            except Exception as e:
                console.print(f"[red]Error in {self.name}: {e}[/red]")
                # If rate limit, wait and retry (Simple backoff)
                if "429" in str(e):
                    time.sleep(10)
                    continue
                return f"Error encountered: {e}"
        
        return "Error: Maximum tool turns exceeded."

### Instantiate the Agents

1. Research Agent: Uses Google Search to find data.

2. Planner Agent: Takes that data and builds the schedule (No search tool, just pure reasoning).

In [27]:
# Instantiate Agents

MODEL_ID = "gemini-2.5-flash-lite" 

# 1. Research Agent
researcher = Agent(
    name="Research Agent",
    model=MODEL_ID,
    system_instruction="""
    You are a Travel Researcher.
    1. Use Google Search to find real-time info.
    2. Focus on: Top 3 attractions, 1 hidden gem, and local food.
    3. Return a detailed, factual list.
    """,
    tools=[google_search_tool],
    tool_map={} # Researcher uses built-in Google Search, no custom map needed
)

# 2. Planner Agent
# We create a map so the Agent knows which function to run
tool_mapping = {
    "currency_converter": currency_converter
}

planner = Agent(
    name="Planner Agent",
    model=MODEL_ID,
    system_instruction="""
    You are a Trip Planner.
    1. Create a day-by-day itinerary based on the research provided.
    2. ALWAYS use the 'currency_converter' tool to show the budget in local currency.
    3. Format the final output in clean Markdown.
    """,
    tools=[currency_tool],
    tool_map=tool_mapping # <--- We pass the map here!
)

console.print(f"[green]Agents Ready & Linked![/green]")

Agents Ready & Linked!

### Session Management and Orchestration

In [28]:
# The "SwiftPlan" Workflow (Orchestration)

def run_swiftplan(destination, duration, interests, budget_usd):
    console.print(f"[bold cyan]✈️ SwiftPlan: {destination} ({duration})[/bold cyan]")
    
    # 1. Research Phase
    task1 = f"Find things to do in {destination} for {interests}. Check weather for next week."
    research_data = researcher.generate_response(task1)
    
    # Pause for realism
    time.sleep(2)
    
    # 2. Planning Phase
    # We explicitly mention the budget to trigger the tool
    task2 = f"Create a {duration} itinerary. The user has a budget of ${budget_usd} USD. Convert this to the local currency."
    
    final_plan = planner.generate_response(task2, context=research_data)
    
    # 3. Output
    console.print("\n" + "="*50)
    console.print(f"[bold green]✅ ITINERARY FOR {destination.upper()}[/bold green]")
    console.print("="*50 + "\n")
    console.print(Markdown(final_plan))

### Testing

In [30]:
# === TEST RUN ===
destination = "Kandy, Sri Lanka"
duration = "3 Days"
interests = "History, Nature, and Fried Rice"
budget_usd = 400

# execute
run_swiftplan(destination, duration, interests, budget_usd)

✈️ SwiftPlan: Kandy, Sri Lanka (3 Days)

🤖 Research Agent is working...

2025-11-25 08:12:11,444 - google_genai.models - AFC is enabled with max remote calls: 10.
2025-11-25 08:12:16,338 - httpx - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-lite:generateContent "HTTP/1.1 200 OK"
2025-11-25 08:12:16,343 - SwiftPlan - Agent Research Agent finished successfully.


🤖 Planner Agent is working...

2025-11-25 08:12:18,943 - httpx - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-lite:generateContent "HTTP/1.1 200 OK"


...

2025-11-25 08:12:26,173 - httpx - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-lite:generateContent "HTTP/1.1 200 OK"
2025-11-25 08:12:26,175 - SwiftPlan - Agent Planner Agent finished successfully.


==================================================

✅ ITINERARY FOR KANDY, SRI LANKA

==================================================

Here is a 3-day itinerary for Kandy, Sri Lanka, with a budget of approximately 116,000 LKR. This plan balances     
historical exploration, nature, and culinary experiences, with consideration for the mixed weather forecast.       


                                    Kandy: A 3-Day Cultural and Natural Escape                                     

Budget: 116,000 LKR                                                                                                

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                     Day 1: Sacred Sites and City Exploration                                      

 • Morning (9:00 AM - 12:00 PM): Begin your day at the Temple of the Sacred Tooth Relic (Sri Dalada Maligawa).     
   Arrive early to witness the morning rituals if possible. Explore the temple complex, soak in the spiritual      
   atmosphere, and admire the intricate architecture. (Entrance Fee: ~1,500 LKR)                                   
 • Lunch (12:00 PM - 1:00 PM): Enjoy your first taste of Kottu Roti at a local eatery near the temple. Opt for a   
   mixed or vegetable kottu to start. (Budget: 500 - 800 LKR)                                                      
 • Afternoon (1:00 PM - 4:00 PM): Visit the Royal Palace of Kandy, including the Royal Audience Hall and the       
   Queen's Bathing Pavilion. Gain insights into the kingdom's past. (Entrance Fee: ~500 LKR)                       
 • Late Afternoon (4:00 PM - 5:30 PM): Take a leisurely stroll around Kandy Lake, enjoying the city views and the  
   serene environment.                                                                                             
 • Evening (6:00 PM onwards): Experience a traditional Kandyan dance performance (optional, ~1,000 LKR) followed by
   dinner at a restaurant offering local Sri Lankan cuisine, perhaps trying some rice and curry. (Budget: 1,500 -  
   2,500 LKR)                                                                                                      

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                    Day 2: Botanical Wonders and Forest Trails                                     

 • Morning (9:00 AM - 12:30 PM): Travel to the Royal Botanic Gardens, Peradeniya. Spend your morning exploring the 
   vast collection of flora, including the famous orchid house and spice gardens. (Entrance Fee: ~2,000 LKR)       
 • Lunch (12:30 PM - 1:30 PM): Have lunch at a cafe within or near the botanic gardens. (Budget: 800 - 1,200 LKR)  
 • Afternoon (2:00 PM - 5:00 PM): Head to the Udawatta Kele Sanctuary. Embark on a hike through the forest trails. 
   Keep an eye out for diverse bird species and enjoy the peaceful natural surroundings. Remember to wear          
   appropriate footwear and carry water. (Entrance Fee: ~500 LKR)                                                  
 • Late Afternoon (5:00 PM - 6:00 PM): If weather permits, find a viewpoint within or near the sanctuary for       
   panoramic views of Kandy.                                                                                       
 • Dinner (7:00 PM onwards): Try another variation of Kottu or explore other Sri Lankan dishes like hoppers (appam)
   or string hoppers (idiappa) for dinner. (Budget: 1,000 - 2,000 LKR)                                             

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                    Day 3: Ancient Temples and Mountain Vistas                                     

 • Morning (8:30 AM - 12:00 PM): Take a trip to Lankatilaka Temple. Admire its unique architecture, stone carvings,
   and the serene ambiance. This offers a different perspective on Buddhist artistry compared to the Temple of the 
   Tooth. (Entrance Fee: ~500 LKR)             